## This program is designed to find the shortest navigation route between any two given cities, using Euclidean distance as a heuristic function to optimize the search. 

In [71]:
# Min Heap data structure, which is a core part in Dijkstra's and Prim's Algorithm. 
# The nodes used are Vertex objects. 
class MinHeap:
    ###
    # Initialization
    def __init__(self):
        self.nodes = [0]
    
    # Add a node to the end of the exisitng min heap, and update the heap so that the 
    # min heap property is satisfied. 
    def addNode(self, vertex):
        self.nodes.append(vertex)
        self.heapifyUp(len(self.nodes) - 1)
    
    # Compares the node's value at a given index with its parent's value, and swap their 
    # positions if the parent's value is greater than the node's value. Then, a recursive 
    # call is executed with the index being the parent's index. This process repeats
    # until the node's value is greater than its parent's value or when the index is 1. 
    def heapifyUp(self, index):
        if index == 1:
            return
        if self.nodes[index].getEstimatedDistance() < self.nodes[index//2].getEstimatedDistance():
            self.swapValue(index, index//2)
            self.heapifyUp(index//2)
        else:
            return
            
    # Compares the node's value at a given index with its children's values. If the node has two
    # children, the child with a lower value is compared with the node's value. Then, their positions
    # are swapped if the node's value is greater than the child's value, and a recursive call is
    # executed with the index being the child's index. This process repeats until the node's value
    # is smaller than its child's value or when the node has no children node. 
    def heapifyDown(self, index):
        if self.isOutOfBound(index):
            return
        if self.isOutOfBound(2 * index):
            return
        if self.isOutOfBound(2 * index + 1):
            j = 2 * index
        else:
            if self.nodes[2 * index + 1].getEstimatedDistance() < self.nodes[2 * index].getEstimatedDistance():
                j = 2 * index + 1
            else:
                j = 2 * index
        if self.nodes[index].getEstimatedDistance() < self.nodes[j].getEstimatedDistance():
            return
        self.swapValue(index, j)
        self.heapifyDown(j)
            
    # Removes the node at index 1, and returns the node's value. After removing, the last node
    # becomes the first node in the heap, and the heap structure is updated.
    def retrieveMin(self):
        if len(self.nodes) > 0:
            low = self.nodes[1]
            self.swapValue(1, len(self.nodes) - 1)
            del self.nodes[-1]
            self.heapifyDown(1)
            return low
        else:
            return None
    
    # Swaps the value of two nodes based on the two given indices. 
    def swapValue(self, pos1, pos2):
        if not (self.isOutOfBound(pos1) and self.isOutOfBound(pos2)):
            temp = self.nodes[pos1]
            self.nodes[pos1] = self.nodes[pos2]
            self.nodes[pos2] = temp
    
    # Checks if the given vertex exists in the heap. 
    def inHeap(self, vertex):
        if vertex in self.nodes:
            return self.nodes.index(vertex)
        else:
            return None
    
    # Checks if the heap is empty.
    def isEmpty(self):
        return len(self.nodes) == 1
    
    # Checks if the given index is out of the range of the heap. 
    def isOutOfBound(self, index):
        if index >= len(self.nodes) or index < 1:
            return True
        else:
            return False
    
    # Outputs information of nodes in the heap for debugging purposes.
    def output(self):
        for i in range(1, len(self.nodes)):
            print(self.nodes[i].getLabel() + "," + str(self.nodes[i].getEstimatedDistance()), end = " ")
        print()
           
    
import math

# Vertex data structure for vertices in an undirected graph
class Vertex:
    
    # Initialization
    def __init__(self, label):
        self.label = label
        self.neighbors = {}
        self.parent = []
        self.estimatedDist = 0
        self.location = (0,0)
        self.pathNumber = 0
    
    # Returns the label/id of this vertex. 
    def getLabel(self):
        return self.label
    
    # Returns the neighbors of this vertex.
    def getNeighbors(self):
        return self.neighbors
    
    # Adds an edge with a given weight between this vertex and the given vertex. 
    def addEdge(self, weight, vertex):
        if weight < 0:
            return
        if vertex not in self.neighbors:
            self.neighbors[vertex] = weight
            vertex.neighbors[self] = weight
    
    # Returns this vertex's parents in the form of a list.  
    def getParent(self):
        return self.parent
    
    # Clear this vertex's list of parents. 
    def resetParent(self):
        self.parent.clear()
    
    # Add the given parent to the list of this vertex's parents. 
    def addParent(self, parent):
        self.parent.append(parent)
    
    # Returns the estimated distance of this vertex from the source. 
    def getEstimatedDistance(self):
        return self.estimatedDist
    
    # Sets the estimated distance of this vertex from the source with the given value. 
    def setEstimatedDistance(self, value):
        self.estimatedDist = value
    
    # Returns the weight of edge connecting this vertex to the given vertex. 
    def getEdgeWeight(self, vertex):
        if vertex not in self.neighbors:
            return 0
        else:
            return self.neighbors[vertex]
    
    # Returns the coordinate of this vertex in the form of a tuple (x,y). 
    def getLocation(self):
        return self.location
    
    # Sets the coordinate of this vertex with the given coordinate.
    def setLocation(self, coordinate):
        self.location = coordinate
    
    # Returns the number of paths leading to this vertex. 
    def getPathNumber(self):
        return self.pathNumber
    
    # Sets the number of paths leading to this vertex with the given value. 
    def setPathNumber(self, value):
        self.pathNumber = value
    
    # Returns the Euclidean distance between this vertex and the given vertex. 
    def getDistance(self, vertex):
        coor1 = self.location
        coor2 = vertex.getLocation()
        return math.sqrt(pow(coor1[0]-coor2[0],2) + pow(coor1[1]-coor2[1],2))


# City planning class
class CityPlan:
    
    # Initialization
    def __init__(self):
        self.graph = []
        self.edgeNum = 0
    
    # Return the undirected graph, which consists of a list of Vertex objects. 
    def getGraph(self):
        return self.graph
    
    # Returns the index of the Vertex in the undirected graph based on the given label/id. 
    def getVertexIndex(self, label):
        for index in range(len(self.graph)):
            if self.graph[index].getLabel() == label:
                return index
        return None
    
    # Read from a text file of a specific format, and construct the undirected graph using 
    # information stored in the file. The file contains information about intersections and
    # roadways. 
    # 
    # An example input file may look like this: 
    # ____________________________________________________________________________________________________
    #        5 4
    #        0 0 0 
    #        1 10 0
    #        2 20 0
    #        3 30 0
    #        4 15 1
    #
    #        0 1
    #        1 2
    #        2 3 
    #        0 4
    #        4 3
    #_____________________________________________________________________________________________________
    # The first line indicates the number of intersections n (in this case, n = 5) and the number 
    # of roadways m (in this case, m = 4). Following the first line, the next n lines contain 
    # intersection identifiers followed by the coordinates of the location of the intersection. 
    # The value of the intersection identifiers is between 0 and n-1. Using the third line as an
    # example, the intersection with id '1' has a coordinate of (10, 0). 
    #
    # Following the intersection information, there is a blank line, which is followed by possibly
    # more than or equal to m roadway information. In each line, there are two numbers, which indicates
    # a roadway (undirected edge) between 2 intersections y and z. Using the first line after the blank 
    # line as an example, there will be a roadway between intersection '0' and intersection '1'. The 
    # distance (weight) of the roadway will be the Euclidean distance between the two intersections, 
    # which takes into account the coordinates of the two intersections. 
    #
    # Note that there may be more than m roadway information lines. This is because the planning agencies
    # may include future roadway information in their files. For analysis related to the current state of 
    # the city roadways, only the first m roadway information will be considered. Also, none of the 
    # intersections will have ids outside the range of [0, n-1]. 
    def readInput(self, fileName):
        with open(fileName, 'r') as file:
            transition = False
            count = 0
            for lines in file:
                lines = lines.split()
                if transition == False:
                    if len(lines) == 0:
                        transition = True
                        continue
                    elif len(lines) == 2:
                        for number in range(int(lines[0])):
                            self.graph.append(Vertex(number))
                        self.edgeNum = int(lines[1])
                    elif len(lines) == 3:
                        label = int(lines[0])
                        index = self.getVertexIndex(label)
                        self.graph[index].setLocation((int(lines[1]), int(lines[2])))
                else:
                    count += 1
                    if count > self.edgeNum:
                        break
                    start = self.graph[self.getVertexIndex(int(lines[0]))]
                    end = self.graph[self.getVertexIndex(int(lines[1]))]
                    distance = start.getDistance(end)
                    start.addEdge(distance, end)
    
    # Dijkstra's algorithm to find shortest path to all other intersections given a starting point. 
    def Dijkstra(self, startID):
        for vertices in self.graph:
            if vertices.getLabel() == startID:
                vertices.setEstimatedDistance(0)
                vertices.setPathNumber(1)
            else:
                vertices.setEstimatedDistance(math.inf)
        minHeap = MinHeap()        
        for element in self.graph:
            minHeap.addNode(element)
        while not minHeap.isEmpty():
            top = minHeap.retrieveMin()
            adjacent = top.getNeighbors()
            for neighbor in adjacent:
                if minHeap.inHeap(neighbor) != None:
                    if neighbor.getEstimatedDistance() > top.getEstimatedDistance() + top.getEdgeWeight(neighbor):
                        neighbor.setEstimatedDistance(top.getEstimatedDistance() + top.getEdgeWeight(neighbor))
                        minHeap.heapifyUp(minHeap.inHeap(neighbor))
                        neighbor.resetParent()
                        neighbor.addParent(top)
                        neighbor.setPathNumber(top.getPathNumber())
                    elif neighbor.getEstimatedDistance() == top.getEstimatedDistance() + top.getEdgeWeight(neighbor):
                        neighbor.addParent(top)
                        neighbor.setPathNumber(neighbor.getPathNumber() + top.getPathNumber())
    
    # A variant of Dijkstra's algorithm that uses adaptive relaxation, where the relaxation depends
    # on two input parameters: A and B, and the target t. It is defined as follows:
    #
    #     d(v) = min{d(v), A * (d(u) + wt(u,v)) + B * (distance(v,t) - distance(u,t))}
    #
    # where wt(u,v) is the Euclidean distance between intersections u and v that are connected by a roadway
    # and distance(u,t) and distance(v,t) are Euclidean distances of target t from u and v, respectively, but
    # it does not necessarily mean that there exist a roadway between intersections u and t, or v and t. 
    def DijkstraVariant(self, startID, endID, A, B):
        variant = 0
        end = self.graph[self.getVertexIndex(endID)]
        for vertices in self.graph:
            if vertices.getLabel() == startID:
                vertices.setEstimatedDistance(0)
                vertices.setPathNumber(1)
            else:
                vertices.setEstimatedDistance(math.inf)
        minHeap = MinHeap()        
        for element in self.graph:
            minHeap.addNode(element)
        while not minHeap.isEmpty():
            top = minHeap.retrieveMin()
            adjacent = top.getNeighbors()
            for neighbor in adjacent:
                if minHeap.inHeap(neighbor) != None:
                    variant = (A * (top.getEstimatedDistance() + top.getEdgeWeight(neighbor))) + (B * (neighbor.getDistance(end) - top.getDistance(end)))
                    if neighbor.getEstimatedDistance() > variant:
                        neighbor.setEstimatedDistance(variant)
                        minHeap.heapifyUp(minHeap.inHeap(neighbor))
                        neighbor.resetParent()
                        neighbor.addParent(top)
                        neighbor.setPathNumber(top.getPathNumber())
                    elif neighbor.getEstimatedDistance() == variant:
                        neighbor.addParent(top)
                        neighbor.setPathNumber(neighbor.getPathNumber() + top.getPathNumber())
    
    # Returns a list containing the shortest path distance to each intersection with the given starting 
    # point i. In the list, the value of the j-th element corresponds to the shortest path distance between
    # intersections i and j. If there is no path to some intersection, then for that intersection, the 
    # shortest path distance is infinity (not reachable). 
    def shortestPathDistance(self, startID):
        self.Dijkstra(startID)
        shortestDistance = []
        for vertex in self.graph:
            shortestDistance.append(round(vertex.getEstimatedDistance()))
        return shortestDistance
     
    # Returns the number of shortest paths between two intersections given the ids of the starting
    # intersection and the destination intersection. 
    def noOfShortestPaths(self, startID, endID):
        self.Dijkstra(startID)
        return self.graph[self.getVertexIndex(endID)].getPathNumber()
    
    # Returns the path from the starting intersection to the destination intersection, given the id of the
    # starting intersection, the id of the destination intersection, and the adaptive relaxation parameters,
    # A and B. If there is no path, 'None' is returned. 
    def fromSrcToDest(self, startID, endID, A, B):
        self.DijkstraVariant(startID, endID, A, B)
        path = []
        start = self.graph[self.getVertexIndex(startID)]
        end = self.graph[self.getVertexIndex(endID)]
        path.append(end.getLabel())
        while end != start:
            if len(end.getParent()) == 0:
                return None
            path.append(end.getParent()[0].getLabel())
            end = end.getParent()[0]
        path.reverse()
        return path
    
    # Returns the path from the starting intersection to the destination intersection with the constraint 
    # that the path in between should contain intersections specified by the user, where startID 
    # corresponds to the id of the starting intersection, endID corresponds to the id of the destination
    # intersection, stopsIDs corresponds to the list of ids of intersections that needs to be visited
    # on the way in order from the starting intersection to the destination intersection and it does 
    # not contain the ids of the starting or destination intersection. A and B corresponds to the adaptive
    # relaxation parameters. If there is no path, 'None' is returned. 
    def fromSrcToDestVia(self, startID, endID, stopsIDs, A, B):
        if len(stopsIDs) == 0 or stopsIDs == None:
            return self.fromSrcToDest(startID, endID, A, B)
        path = [startID]
        dest = None
        while dest != endID:
            if len(stopsIDs) > 0:
                dest = stopsIDs.pop(0)
            else:
                dest = endID
            nextPath = self.fromSrcToDest(startID, dest, A, B)[1::]
            if nextPath != None:
                path += nextPath
            else:
                return None
            startID = dest
        return path
        
            
    # Prim's algorithm to find the minimal spanning tree from a graph. 
    def primAlgo(self, startID):
        for vertices in self.graph:
            if vertices.getLabel() == startID:
                vertices.setEstimatedDistance(0)
                vertices.addParent(vertices)
            else:
                vertices.setEstimatedDistance(math.inf)
        minHeap = MinHeap()        
        for element in self.graph:
            minHeap.addNode(element)
        while not minHeap.isEmpty():
            top = minHeap.retrieveMin()
            adjacent = top.getNeighbors()
            for neighbor in adjacent:
                if minHeap.inHeap(neighbor) != None:
                    if neighbor.getEstimatedDistance() > top.getEdgeWeight(neighbor):
                        neighbor.setEstimatedDistance(top.getEdgeWeight(neighbor))
                        neighbor.resetParent()
                        neighbor.addParent(top)
                        minHeap.heapifyUp(minHeap.inHeap(neighbor))
    
    # Computes the minimum cost reachability tree from a starting intersection given its id. This method 
    # returns a list, where the i-th element in the list is the id of the parent of the intersection with 
    # id i. If the id of the source is i, then the i-th element in the list is also i. If i is the id of an
    # intersection that is not reachable from the starting intersection, then the i-th element in the list
    # is equal to -1. 
    def minCostReachabilityFromSrc(self, startID):
        self.primAlgo(startID)
        array = []
        for element in self.graph:
            if len(element.getParent()) == 0:
                array.append(-1)
            else:
                array.append(element.getParent()[0].getLabel())
        return array
    
    # Returns the cost of the minimum cost reachability tree from a intersection given its id. 
    def minCostOfReachabilityFromSrc(self, startID):
        self.primAlgo(startID)
        totalMinCost = 0
        for element in self.graph:
            if len(element.getParent()) != 0:
                parent = element.getParent()[0]
                totalMinCost += element.getEdgeWeight(parent)
        return round(totalMinCost)
    
    # Returns True if all intersections are reachable from the starting intersection given its id. Returns 
    # False if there is at least one intersection that is not reachable from the starting intersection. 
    def isFullReachableFromSrc(self, startID):
        reachabilityTree = self.minCostReachabilityFromSrc(startID)
        if -1 in reachabilityTree:
            return False
        else:
            return True